In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46302
2,Huelva,Confirmados PDIA 14 días,389
3,Huelva,Tasa PDIA 14 días,"75,8033400237738"
4,Huelva,Confirmados PDIA 7 días,180
5,Huelva,Tasa PDIA 7 dias,"35,07609564082078"
6,Huelva,Total Confirmados,46506
7,Huelva,Curados,45192
8,Huelva,Fallecidos,420


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46302.0


/tmp/ipykernel_145381/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12720.0


/tmp/ipykernel_145381/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_145381/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_145381/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_145381/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3425 personas en los últimos 7 días 

Un positivo PCR cada 1635 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46302.0,180.0,389.0,513170.0,35.076096,75.803340,34.0
Huelva-Costa,27382.0,93.0,221.0,289548.0,32.119027,76.325860,17.0
Condado-Campiña,14393.0,74.0,132.0,156231.0,47.365760,84.490274,12.0
Cartaya,2144.0,28.0,64.0,20083.0,139.421401,318.677488,8.0
Huelva (capital),12720.0,42.0,88.0,143837.0,29.199719,61.180364,5.0
Palos de la Frontera,1081.0,14.0,26.0,11742.0,119.230114,221.427355,5.0
Sierra de Huelva-Andévalo Central,4100.0,13.0,35.0,67391.0,19.290410,51.935718,5.0
Moguer,2040.0,25.0,38.0,21867.0,114.327525,173.777839,4.0
Aljaraque,1646.0,8.0,13.0,21474.0,37.254354,60.538325,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,53.0,3.0,18.0,713.0,420.757363,2524.544180,0.0
Cartaya,2144.0,28.0,64.0,20083.0,139.421401,318.677488,8.0
San Bartolomé de la Torre,388.0,6.0,10.0,3761.0,159.532039,265.886732,0.0
Palos de la Frontera,1081.0,14.0,26.0,11742.0,119.230114,221.427355,5.0
Minas de Riotinto,188.0,5.0,7.0,3812.0,131.164743,183.630640,0.0
Moguer,2040.0,25.0,38.0,21867.0,114.327525,173.777839,4.0
Almonaster la Real,87.0,1.0,2.0,1788.0,55.928412,111.856823,1.0
Isla Cristina,3168.0,9.0,21.0,21393.0,42.069836,98.162950,1.0
Almonte,2332.0,9.0,22.0,24507.0,36.724201,89.770270,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Campofrío,53.0,3.0,18.0,713.0,420.757363,2524.544180,0.0,0.166667
Lepe,3048.0,2.0,12.0,27880.0,7.173601,43.041607,1.0,0.166667
Rociana del Condado,674.0,1.0,5.0,7939.0,12.596045,62.980224,0.0,0.200000
Ayamonte,2112.0,3.0,13.0,21104.0,14.215315,61.599697,0.0,0.230769
Palma del Condado (La),1436.0,2.0,6.0,10801.0,18.516804,55.550412,0.0,0.333333
Sierra de Huelva-Andévalo Central,4100.0,13.0,35.0,67391.0,19.290410,51.935718,5.0,0.371429
Almonte,2332.0,9.0,22.0,24507.0,36.724201,89.770270,1.0,0.409091
Huelva-Costa,27382.0,93.0,221.0,289548.0,32.119027,76.325860,17.0,0.420814
Isla Cristina,3168.0,9.0,21.0,21393.0,42.069836,98.162950,1.0,0.428571
